In [2]:
from concurrent.futures import ThreadPoolExecutor
import numpy as np
from time import sleep
import csv

def generate_fake_coordinates(n, lat_range, lon_range):
    """
    Genera una matriz de n pares de latitudes y longitudes aleatorias en los rangos dados.
    Los rangos de latitud y longitud deben ser pares de números en el formato (min, max).
    """
    lat_min, lat_max = lat_range
    lon_min, lon_max = lon_range
    
    latitudes = np.random.uniform(lat_min, lat_max, n)
    longitudes = np.random.uniform(lon_min, lon_max, n)
    
    return np.column_stack((latitudes, longitudes))

def fake_request(lat1, lon1, lat2, lon2):
    # Random sleep
    sleep(np.random.randint(0, 100) / 100)
    return abs(lat2-lat1) + abs(lon1 - lon2), np.random.randint(0, 1000)


points = generate_fake_coordinates(10, (-33.4569, 40.45678), (-40.7580, -33.4569))
points2 = generate_fake_coordinates(10, (-33.4569, 40.45678), (-40.7580, -33.4569))

lat_s = list(points[:, 0])
long_s = list(points[:, 1])

lat_d = list(points2[:, 0])
long_d = list(points2[:, 1])

def slow_imp():
    for (lat_s_i, long_s_i, lat_d_j, long_d_j) in zip(lat_s, long_s, lat_d, long_d):
        dist, time = fake_request(lat_s_i, long_s_i, lat_d_j, long_d_j)
        print("Distance:", dist)
        print("Time: ", time)



In [3]:
%time slow_imp()

Distance: 16.370365848593124
Time:  836
Distance: 32.72899007883553
Time:  619
Distance: 41.627310289613696
Time:  66
Distance: 5.381903083480246
Time:  7
Distance: 4.588306009310202
Time:  470
Distance: 33.28725723938799
Time:  319
Distance: 30.876157692142943
Time:  849
Distance: 58.87001367114716
Time:  637
Distance: 7.4599350344711155
Time:  729
Distance: 3.63867052442707
Time:  937
CPU times: total: 15.6 ms
Wall time: 5.1 s


In [4]:


def fast_imp():
    # Ejecutamos en paralelo 
    with ThreadPoolExecutor() as excecutor:
        # Guardamos los futures
        future = {excecutor.submit(fake_request,lat_s_i, long_s_i, lat_d_j, long_d_j) for lat_s_i, long_s_i, lat_d_j, long_d_j in  zip(lat_s, long_s, lat_d, long_d)}
        for i in future:
            distance, time = i.result()
            print("Distance: ", distance)
            print("Time ", time)

def fast_imp_refactor(lat_s, long_s, lat_d, long_d, from_backup=True):
    futures = []
    with ThreadPoolExecutor() as excecutor:
        for lat_s_i, long_s_i, lat_d_j, long_d_j in  zip(lat_s, long_s, lat_d, long_d):
            futures.append(excecutor.submit(fake_request, lat_s_i, long_s_i, lat_d_j, long_d_j))
        for future in futures:
            time, dst = future.result()
            print("Distance: ", dst)
            print("Time ", time)

            


points = generate_fake_coordinates(100, (-33.4569, 40.45678), (-40.7580, -33.4569))
points2 = generate_fake_coordinates(10, (-33.4569, 40.45678), (-40.7580, -33.4569))

lat_s = list(points[:, 0])
long_s = list(points[:, 1])

lat_d = list(points2[:, 0])
long_d = list(points2[:, 1])




import functools
import itertools
from concurrent.futures import ThreadPoolExecutor

@functools.lru_cache(maxsize=1_000_000_000)
def cached_request(lat_s, lon_s, lat_d, lon_d):
    sleep(np.random.randint(0, 100) / 100)
    # Haz lo de request y eso que hacen las requests de forma sincrona

    # Retorna el resultado 
    return abs(lat_d-lat_s) + abs(lon_d - lon_s), np.random.randint(0, 1000)

def fast_imp_refactor(lat_s, long_s, lat_d, long_d):
    futures = []
    with ThreadPoolExecutor() as executor:
        p = itertools.product(zip(lat_s, long_s), zip(lat_d, long_d))

        print(f"Testing w/{len(lat_s)*len(long_d)} Points")
        for cords in itertools.product(zip(lat_s, long_s), zip(lat_d, long_d)):
            lat_s_i, long_s_i = cords[0]
            lat_s_j, long_d_j = cords[1]
            futures.append(executor.submit(cached_request, lat_s_i, long_s_i, lat_s_j, long_d_j))

        for future in futures:
            time, dst = future.result()
            print("Distance: ", dst)
            print("Time ", time)



In [5]:
%time fast_imp_refactor(lat_s, long_s, lat_d, long_d)

Testing w/1000 Points
Distance:  987
Time  41.945567522774084
Distance:  19
Time  10.5185149422072
Distance:  797
Time  8.366623656424952
Distance:  537
Time  49.114063259845125
Distance:  962
Time  41.50122945246197
Distance:  342
Time  41.00150873370088
Distance:  822
Time  23.531360357876945
Distance:  14
Time  4.605288841234142
Distance:  850
Time  7.216927237057785
Distance:  200
Time  5.5180641226611975
Distance:  232
Time  24.556264403253024
Distance:  821
Time  11.541934085370094
Distance:  224
Time  29.61122710059967
Distance:  832
Time  33.070223860861084
Distance:  639
Time  25.45739005347793
Distance:  448
Time  24.95766933471684
Distance:  409
Time  1.47091133029965
Distance:  165
Time  17.455160186343154
Distance:  767
Time  29.27737626463508
Distance:  921
Time  16.542384904916098
Distance:  435
Time  48.91103163860235
Distance:  968
Time  17.81179572121243
Distance:  87
Time  7.471011959301141
Distance:  505
Time  57.42499109621042
Distance:  34
Time  49.812157288827265

Distance:  4
Time  10.627504460361848
Distance:  415
Time  15.541299800907353
Distance:  204
Time  24.08559423984213
Distance:  379
Time  19.537833051911655
Distance:  367
Time  16.30862087629194
Distance:  888
Time  48.21194345575186
Distance:  554
Time  45.70400641996429
Distance:  786
Time  7.411400584313949
Distance:  840
Time  22.4525624053072
Distance:  168
Time  6.767906857029008
CPU times: total: 0 ns
Wall time: 913 ms


In [ ]:
from cachetools import TTLCache

def make_request(lat_s, long_s, lat_d, long_d, cache):
    cache_key = (lat_s, long_s, lat_d, long_d)
    if cache_key in cache:
        duration, distance = cache[cache_key]
        print(f"Distance from ({lat_s}, {long_s}) to ({lat_d}, {long_d}): {distance}")
        print(f"Time from ({lat_s}, {long_s}) to ({lat_d}, {long_d}): {duration}")
        return duration, distance
    else:
        url = f"http://router.project-osrm.org/route/v1/driving/{long_s},{lat_s};{long_d},{lat_d}"
        response = requests.get(url)
        data = response.json()
        distance = data["routes"][0]["distance"]
        duration = data["routes"][0]["duration"]
        cache[cache_key] = (duration, distance)
        print(f"Distance from ({lat_s}, {long_s}) to ({lat_d}, {long_d}): {distance}")
        print(f"Time from ({lat_s}, {long_s}) to ({lat_d}, {long_d}): {duration}")
        return duration, distance